In [1]:
import numpy as np
import pandas as pd
import glob #, ipyplot
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFilter

basepath = '/home/ec2-user/hubmap-hacking-the-human-vasculature'

train = pd.read_csv(f'{basepath}/tile_meta.csv')
train['path'] = train['id'].map(lambda x: basepath + 'train/' + str(x) + '.tif')
wsi = pd.read_csv(p+'wsi_meta.csv')
mask = pd.read_json(p+'polygons.jsonl', lines=True)

test = pd.DataFrame(glob.glob(p+'test/**.tif'), columns=['path'])
test['id'] = test['path'].map(lambda x: x.split('/')[-1].split('.')[0])

sub = pd.read_csv(p+'sample_submission.csv')

In [2]:
train_annot = pd.merge(train, mask, on='id', how='inner')
dataset1_ids = set(train_annot.loc[train_annot['dataset']==1, 'id'].values)
dataset2_ids = set(train_annot.loc[train_annot['dataset']==2, 'id'].values)

In [ ]:
print(train[wsi.source_wsi == 1])
paths = train[train['source_wsi']==2].reset_index(drop=True)

x, y = (paths.i.max() + 512, paths.j.max() + 512)
x_, y_ = paths.i.min(), paths.j.min()
img = Image.new(mode="RGB", size=(x - x_,y - y_)) #'TIFF'
draw = ImageDraw.Draw(img)
tgt_ids = set(['6a4d868ab872'])
for i in range(len(paths)):
    img.paste(Image.open(paths.path[i]), (paths.i[i] - x_, paths.j[i] - y_))
    if paths.id[i] in tgt_ids:
        draw.rectangle([(paths.i[i] - x_, paths.j[i] - y_), (paths.i[i] - x_ + 512, paths.j[i] - y_ + 512)], outline="green", width=50)
    else:
        if paths.id[i] in dataset1_ids:
            draw.rectangle([(paths.i[i] - x_, paths.j[i] - y_), (paths.i[i] - x_ + 512, paths.j[i] - y_ + 512)], outline="red", width=50)
        if paths.id[i] in dataset2_ids:
            draw.rectangle([(paths.i[i] - x_, paths.j[i] - y_), (paths.i[i] - x_ + 512, paths.j[i] - y_ + 512)], outline="blue", width=50)
    #break
img = img.resize((x // 2, y // 2))
plt.imshow(img)

In [ ]:
sub.to_csv('submission.csv', index=False)